# Create Lidar Derived Contours from USGS Lidar Data in ArcGIS Pro
#### Author:
Langdon Sanders | GIS@dublin.oh.us
December, 2023

# Sections
* Download .laz files
* Convert them to .las
* Create a Las Dataset
* Create a Filter for Ground Only Points
* Create a Digital Elevation Model (DEM)
* Create Contours from DEM
* Export Contours as Shapefile

# Download .laz files from list of URLs, i.e. USGS `data.txt`

## Purpose

* Lidar data is often provided in many tiles due to its large size, this is the case with data from USGS 3DEp program.
* Users create a "cart" of data to download from the USGS website, https://www.usgs.gov/the-national-map-data-delivery/gis-data-download
* There, a `data.txt` file can be downloaded with a list of all each URL containing the zipped lidar tile from the cart. 

Example `data.txt` contents
```
https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785753.laz
https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785755.laz
https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785756.laz
https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785757.laz
https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785758.laz
https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785760.laz
```

This portion of the notebook will use the `requests` library to download each of these files to a specified folder

## Gather and clean list of data source URLs from file

In [28]:
import os

In [2]:
# print current working directory
os.getcwd()

'G:\\GIS Data\\Users Planning\\Projects\\MetroCenterProject'

In [18]:
# if the data.txt file is in this file you can simply reference "data.txt" as the filename
# otherwise, use the full filepath 

lidarURLFile = open("data.txt", "r")

# create a list of each line in the file
urls = lidarURLFile.readlines()

# print first 5 for review
urls[:5]

['https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785753.laz\n', 'https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785755.laz\n', 'https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785756.laz\n', 'https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785757.laz\n', 'https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785758.laz\n']

Remove the `\n` from each line that is in the original data.txt file

In [27]:
# remove the \n in each line https://stackoverflow.com/questions/7984169/remove-trailing-newline-from-the-elements-of-a-string-list 
urls = list([url.strip() for url in urls])
urls[:5]

['https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785753.laz', 'https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785755.laz', 'https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785756.laz', 'https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785757.laz', 'https://rockyweb.usgs.gov/vdelivery/Datasets/Staged/Elevation/LPC/Projects/OH_Columbus_2019_B19/OH_Columbus_2019/LAZ/USGS_LPC_OH_Columbus_2019_B19_BS785758.laz']

# Download Files
for each line, download the `.laz` file to a folder

### Function to create download folder if does not exist

In [72]:
def create_download_folder(download_folder = "lidar"):
    # download_folder folder for the .las files

    # create the folder if it doesn't exist
    if not os.path.exists(download_folder):

        # make the folder
        print("Ope, looks like that folder doesn't exist yet, no worries, I'll make that now.")
        os.makedirs(download_folder)

        # check it's made
        print("\t Does it exist now? ", os.path.exists(download_folder))

### Using a template code from Real Python blog to download in batch
https://realpython.com/python-download-file-from-url/

> This function takes a URL as an argument, makes a GET request using the requests library, and saves the retrieved data in a local file. In this specific example, it first attempts to extract the filename from the Content-Disposition response header, which contains information on which items on the page are displayed inline versus as attachments. If that’s not available, then it gets the filename from a part of the URL

In [41]:
from concurrent.futures import ThreadPoolExecutor
import requests

In [74]:
# source: https://realpython.com/python-download-file-from-url/ 
# Edits for creating download destionation folder

# can name the downlaod folder whatever you want, will be created in current working directory 

def download_file(url, download_folder="lidar"): 
    
    #create location if does nto exist
    create_download_folder(download_folder)
    
    response = requests.get(url)
    if "content-disposition" in response.headers:
        content_disposition = response.headers["content-disposition"]
        filename = content_disposition.split("filename=")[1]
    else:
        filename = url.split("/")[-1]
    print(filename)

    
    # download to specified folder location
    with open(os.path.join(download_folder, filename), mode="wb") as file:
        file.write(response.content)
        
    print(f"Downloaded file {os.path.abspath(filename)}")

In [76]:
with ThreadPoolExecutor() as executor:
    executor.map(download_file, urls)

print("Downlaods Complete")

USGS_LPC_OH_Columbus_2019_B19_BS786757.laz
Downloaded file G:\GIS Data\Users Planning\Projects\MetroCenterProject\USGS_LPC_OH_Columbus_2019_B19_BS786757.laz
USGS_LPC_OH_Columbus_2019_B19_BS786755.laz
Downloaded file G:\GIS Data\Users Planning\Projects\MetroCenterProject\USGS_LPC_OH_Columbus_2019_B19_BS786755.laz
USGS_LPC_OH_Columbus_2019_B19_BS785753.laz
Downloaded file G:\GIS Data\Users Planning\Projects\MetroCenterProject\USGS_LPC_OH_Columbus_2019_B19_BS785753.laz
USGS_LPC_OH_Columbus_2019_B19_BS785755.laz
Downloaded file G:\GIS Data\Users Planning\Projects\MetroCenterProject\USGS_LPC_OH_Columbus_2019_B19_BS785755.laz
USGS_LPC_OH_Columbus_2019_B19_BS786762.laz
Downloaded file G:\GIS Data\Users Planning\Projects\MetroCenterProject\USGS_LPC_OH_Columbus_2019_B19_BS786762.laz
USGS_LPC_OH_Columbus_2019_B19_BS786753.laz
Downloaded file G:\GIS Data\Users Planning\Projects\MetroCenterProject\USGS_LPC_OH_Columbus_2019_B19_BS786753.laz
USGS_LPC_OH_Columbus_2019_B19_BS786765.laz
Downloaded file

USGS_LPC_OH_Columbus_2019_B19_BS793765.laz
USGS_LPC_OH_Columbus_2019_B19_BS795756.laz
Downloaded file G:\GIS Data\Users Planning\Projects\MetroCenterProject\USGS_LPC_OH_Columbus_2019_B19_BS795756.laz
Downloaded file G:\GIS Data\Users Planning\Projects\MetroCenterProject\USGS_LPC_OH_Columbus_2019_B19_BS793765.laz
USGS_LPC_OH_Columbus_2019_B19_BS795767.laz
Downloaded file G:\GIS Data\Users Planning\Projects\MetroCenterProject\USGS_LPC_OH_Columbus_2019_B19_BS795767.laz
USGS_LPC_OH_Columbus_2019_B19_BS793768.laz
USGS_LPC_OH_Columbus_2019_B19_BS793766.laz
Downloaded file G:\GIS Data\Users Planning\Projects\MetroCenterProject\USGS_LPC_OH_Columbus_2019_B19_BS793768.laz
Downloaded file G:\GIS Data\Users Planning\Projects\MetroCenterProject\USGS_LPC_OH_Columbus_2019_B19_BS793766.laz
USGS_LPC_OH_Columbus_2019_B19_BS795768.laz
Downloaded file G:\GIS Data\Users Planning\Projects\MetroCenterProject\USGS_LPC_OH_Columbus_2019_B19_BS795768.laz
USGS_LPC_OH_Columbus_2019_B19_BS793753.laz
Downloaded file

# Convert a folder of lidar `.laz` files to `.las` in batch

## Requirements

* folder with `.laz` files, i.e. downloaded in previous section

## Purpose
* Lidar point cloud data is often provided in `.laz` files; such as from the USGS. Esri-ware works best with `.las` file formats
* Often many lidar point cloud tiles are needed to be merged for the area of interest.
* This process will run the ConvertLas() `arcpy` method on a folder containing `.laz` files, and convert each to `.las` in a destination folder.

#### General steps
* define origin folder containing .laz files
* define destination folder for .las files in current working directory
* create a list of all .laz files in origin folder
* iterate on this list and convert each to .las sent to the destination folder.

## Define origin and destination folders (create if needed)

### You likely created this folder in the previous step

i.e. this is a folder named "lidar" in the current working directory, but can specify it otherwise

In [91]:
# def origin folder containing .laz files in the cwd

# In this c
org_fold = os.path.join(os.getcwd(), "lidar")
print("origin folder of .laz to convert: ", org_fold)

origin folder of .laz to convert:  G:\GIS Data\Users Planning\Projects\MetroCenterProject\lidar


In [89]:
# destination folder for the .las files
dest_fold = "LidarLas"

# create the destination folder if it doesn't exist
if not os.path.exists(dest_fold):
    
    # make the folder
    print("Ope, looks like that folder doesn't exist yet, no worries, I'll make that now.")
    os.makedirs(dest_fold)
    
    # check it's made
    print("\t Does it exist now? ", os.path.exists(dest_fold))
print("destination of .las files: ", os.path.abspath(dest_fold))

destination of .las files:  G:\GIS Data\Users Planning\Projects\MetroCenterProject\LidarLas


## Get list of `.laz` files in folder

In [132]:
laz_files = []

for f in os.listdir(org_fold):
    if f.endswith(".laz"):
        laz_files.append(f)
        print(f, " added to file list")
        
tot_files =   len(laz_files)      
print("\nTotal files found: ",tot_files )

USGS_LPC_OH_Columbus_2019_B19_BS785753.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS785755.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS785756.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS785757.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS785758.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS785760.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS785761.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS785762.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS785763.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS785765.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS785766.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS785767.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS785768.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS786753.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS786755.laz  added to file list
USGS_LPC_OH_Columbus_2019_B19_BS786756.laz  added to fi

### Some error handling for future runs

When running this with large number of files, i.e. 207, it got through 200 of them then crashed, so adding some checking first to see if files are already in the destionation folder so you can rerun this code and only reprocess those that were not done the first time around

In [133]:
# check if files already exist; skip those.
# this is useful if started a large batch and didn't work on all of them.
files_done = []

for f in os.listdir(dest_fold):
    if f.endswith(".las"):
        files_done.append(f)
        #print(f, " added to file done list")
        
tot_files_done =   len(files_done)
print("\nFiles in original total list", tot_files)
print("\nTotal files done: ",tot_files_done )


Files in original total list 208

Total files done:  207


### Create a Files To Do list
* Compare **files done vs original list**

In [134]:
# remove file extensions (.laz and .las) to compare the file names form origin to done list.
laz_no_ext = [os.path.splitext(x)[0] for x in laz_files]
files_done_no_ext = [os.path.splitext(x)[0] for x in files_done]

# use set functions to compare and find missing
not_yet = set(laz_no_ext) - set(files_done_no_ext)

# add file extension back on and listify the set
laz_to_do = [x + ".laz" for x in list(not_yet)]

print("LAZ files that have not yet been converted:\n ", laz_to_do)

LAZ files that have not yet been converted:
  ['USGS_LPC_OH_Columbus_2019_B19_BS787761.laz']


# Convert list of `.laz` files to `.las`

In [135]:
# lookin gat what is left to go, printing the list
laz_to_do

['USGS_LPC_OH_Columbus_2019_B19_BS787761.laz']

In [136]:
# checking the aboslute path of the destinatoin folder
os.path.abspath(dest_fold)

'G:\\GIS Data\\Users Planning\\Projects\\MetroCenterProject\\LidarLas'

In [137]:
org_fold

'G:\\GIS Data\\Users Planning\\Projects\\MetroCenterProject\\lidar'

## Template Code

### Copied ArcPy tool GUI for convert Las

```
arcpy.conversion.ConvertLas(
    in_las=r"G:\GIS Data\Users Planning\Projects\MetroCenterProject\Lidar\USGS_LPC_OH_Columbus_2019_B19_BS786760.laz",
    target_folder=r"G:\GIS Data\Users Planning\Projects\MetroCenterProject\Lidar",
    file_version="SAME_AS_INPUT",
    point_format="",
    compression="NO_COMPRESSION",
    las_options="REARRANGE_POINTS",
    out_las_dataset=None,
    define_coordinate_system="NO_FILES",
    in_coordinate_system=None
)
```

In [138]:
# iterate all files in list of laz_files
for i,f in enumerate(laz_to_do):
    print("file ", i+1, " / ", len(laz_to_do))

    print("Running arcpy.conversion.ConvertLas() on ", f, " sending to ", dest_fold)
    arcpy.conversion.ConvertLas(
    
    # define the file being processed, by origin folder and file name
    in_las= os.path.join(org_fold, f),
        
    # destination folder 
    target_folder= dest_fold,
        
    file_version="SAME_AS_INPUT",
    point_format="",
    compression="NO_COMPRESSION",
    las_options="REARRANGE_POINTS",
    out_las_dataset=None,
    define_coordinate_system="NO_FILES",
    in_coordinate_system=None)
          
    print("\t\tcomplete")
print("All Done")

file  1  /  1
Running arcpy.conversion.ConvertLas() on  USGS_LPC_OH_Columbus_2019_B19_BS787761.laz  sending to  LidarLas
		complete
All Done


# Create LAS Dataset and Create Raster DEM

## Purpose:
* Combining `.las` tiles into one dataset allows for creating large area models
* Esri's format for this is called a LAS Dataset, will create this from the `.las` tiles
* Then Create a raster **Digital Elevation Model (DEM)** from the lidar point cloud data as a precursor to creating contours.
* By going to a raster format first, each cell averages the lidar data within its extent which smooths out the noise of lidar

Source docs https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/create-las-dataset.htm 


In [139]:
arcpy.management.CreateLasDataset(
    input=r"'G:\GIS Data\Users Planning\Projects\MetroCenterProject\LidarLas'",
    out_las_dataset=os.path.join(os.getcwd(),"LasDataset.lasd"), # define name and location of destination LAS Dataset
    folder_recursion="NO_RECURSION",
    in_surface_constraints=None,
    spatial_reference=None, # will use the first one to define spatial ref
    compute_stats="COMPUTE_STATS",
    relative_paths="ABSOLUTE_PATHS",
    create_las_prj="NO_FILES",
    extent="DEFAULT",
    boundary=None,
    add_only_contained_files="INTERSECTED_FILES"
)

<Result 'G:\\GIS Data\\Users Planning\\Projects\\MetroCenterProject\\LasDataset.lasd'>

# Make Las Dataset Layer (Filter for Ground Only)

### This creates a layer in the current map which is used by the next step; it is not saving it out to a GIS file.

It requires an active Map in ArcGIS Pro

In [140]:
arcpy.management.MakeLasDatasetLayer(
    in_las_dataset="LasDataset.lasd",
    out_layer="LasDataset_Ground",
    class_code="2",
    return_values=None,
    no_flag="INCLUDE_UNFLAGGED",
    synthetic="INCLUDE_SYNTHETIC",
    keypoint="INCLUDE_KEYPOINT",
    withheld="EXCLUDE_WITHHELD",
    surface_constraints=None,
    overlap="INCLUDE_OVERLAP"
)

<Result 'LasDataset_Ground'>

# Create DEM

In [141]:
with arcpy.EnvManager(cellSize="MAXOF"):
    arcpy.conversion.LasDatasetToRaster(
        in_las_dataset="LasDataset_Ground",
        out_raster=r"G:\GIS Data\Users Planning\Projects\MetroCenterProject\DEM\las_to_dem_2ft.tif",
        value_field="ELEVATION",
        interpolation_type="BINNING AVERAGE LINEAR",
        data_type="FLOAT",
        sampling_type="CELLSIZE",
        sampling_value=2,
        z_factor=1
    )

# Create Contours

In [142]:
arcpy.ddd.Contour(
    in_raster="las_to_dem_2ft.tif",
    out_polyline_features=r"G:\GIS Data\Users Planning\Projects\MetroCenterProject\MetroCenterProject.gdb\Contour_dem2ft",
    contour_interval=1,
    base_contour=0,
    z_factor=1,
    contour_type="CONTOUR",
    max_vertices_per_feature=None
)

<Result 'G:\\GIS Data\\Users Planning\\Projects\\MetroCenterProject\\MetroCenterProject.gdb\\Contour_dem2ft'>

# Create Shapefile of contours for portability to others

In [ ]:
arcpy.conversion.FeatureClassToShapefile(
    Input_Features="Contour_dem2ft",
    Output_Folder=r"G:\GIS Data\Users Planning\Projects\MetroCenterProject\Exports\Shapefile"
)